In [1]:
# #ResNet 50
# from keras.applications.resnet50 import ResNet50
# from keras.preprocessing import image
# from keras.applications.resnet50 import preprocess_input, decode_predictions
# import numpy as np

# model = ResNet50(weights='imagenet')

# img_path = 'cat.jpg'
# img = image.load_img(img_path, target_size=(224,224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# print('Predicted:', decode_predictions(preds, top=3)[0])

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=False, download=False, transform=transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = {'plane', 'car', 'bird', 'cat', 'deer',
          'dog', 'frog', 'horse', 'ship', 'truck'}

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

In [6]:
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
print('Finished Training')

[1,  2000] loss: 2.271
[1,  4000] loss: 1.931
[1,  6000] loss: 1.724
[1,  8000] loss: 1.612
[1, 10000] loss: 1.537
[1, 12000] loss: 1.493
[2,  2000] loss: 1.451
[2,  4000] loss: 1.386
[2,  6000] loss: 1.374
[2,  8000] loss: 1.354
[2, 10000] loss: 1.320
[2, 12000] loss: 1.299
[3,  2000] loss: 1.244
[3,  4000] loss: 1.257
[3,  6000] loss: 1.226
[3,  8000] loss: 1.224
[3, 10000] loss: 1.204
[3, 12000] loss: 1.228
[4,  2000] loss: 1.156
[4,  4000] loss: 1.146
[4,  6000] loss: 1.171
[4,  8000] loss: 1.157
[4, 10000] loss: 1.163
[4, 12000] loss: 1.113
[5,  2000] loss: 1.075
[5,  4000] loss: 1.099
[5,  6000] loss: 1.105
[5,  8000] loss: 1.107
[5, 10000] loss: 1.081
[5, 12000] loss: 1.095
Finished Training


In [9]:
# PATH = './cifar_net.path'
# torch.save(model.state_dict(), PATH)

In [8]:
#test
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %(
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %
